In [ ]:
from collections import namedtuple
import numpy as np
import pandas as pd
import torch
import re

In [ ]:
cudaFile = '/root/MLPerf-mGPU/image_classification/resnet50v15-batch-nv-1042s877m.csv'
cudadf = pd.read_csv(cudaFile, header=0, index_col=0)
display(cudadf)
rocmFile = '/root/MLPerf-mGPU/image_classification/resnet50v15-batch-rocm30-1386s877m.csv'
rocmdf = pd.read_csv(rocmFile, header=0, index_col=0)

# CHECK SHAPE IS SAME
#hack to fix mismatch in total number of rows
rocmdf.reset_index(inplace=True)
rocmdf['joinID'] = rocmdf['joinID'] + 1
rocmdf.set_index('joinID', inplace=True)

display(rocmdf)

In [ ]:
joindf = pd.merge(cudadf, rocmdf, left_index=True, right_index=True, suffixes=('_cuda', '_rocm'))
display(joindf)

In [ ]:
# ops = joindf.groupby('pyName', as_index=False).size().to_frame()
# ops.reset_index(inplace=True)
# ops.rename(columns={0:'count'}, inplace=True)
ops = joindf.groupby(['pyName', 'kernelNames_cuda', 'numKernels_cuda', 'kernelNames_rocm', 'numKernels_rocm'], as_index=False).agg({
    'sumKernelDuration': ['count', 'mean'],
    'duration_cuda': 'mean',
    'kernelDuration': 'mean',
    'duration_rocm': 'mean'
})
# print(ops.columns)
ops.columns = pd.MultiIndex.from_tuples([('pytorch','pyName'), ('cuda','kernelNames'), ('cuda','numKernels'), ('rocm','kernelNames'), ('rocm','numKernels'), ('pytorch','numRepeats'), ('cuda','kernelDuration'), ('cuda','duration'), ('rocm','kernelDuration'), ('rocm','duration')])
ops = ops[[('pytorch','pyName'),('pytorch','numRepeats'),('cuda','kernelNames'), ('cuda','numKernels'),('cuda','kernelDuration'), ('cuda','duration'),('rocm','kernelNames'), ('rocm','numKernels'),('rocm','kernelDuration'), ('rocm','duration')]]
display(ops)
# ops[ops['pyName'].str.startswith("conv2d")]

In [ ]:
# joindf[joindf['pyName'].str.startswith("conv2d")]['duration_rocm'].sum()
ops[('rocm','repsDuration')] = ops[('rocm','duration')] * ops[('pytorch','numRepeats')]
batchTime = ops[('rocm','repsDuration')].sum()
print(batchTime)
ops[('rocm','%BatchTime')] = (ops[('rocm','repsDuration')] / batchTime) * 100
display(ops.sort_values(('rocm', '%BatchTime'), ascending=False).head(50))

In [ ]:
ops[('improve', 'ratioKernelDuration')] = ops[('rocm', 'kernelDuration')] / ops[('cuda', 'kernelDuration')]
ops[('improve', 'rationDuration')] = ops[('rocm', 'duration')] / ops[('cuda', 'duration')]
ops[('improve', 'timeDiff')] = ops[('rocm', 'duration')] - ops[('cuda', 'duration')]
ops[('improve', 'repsTimeDiff')] = ops[('improve', 'timeDiff')] * ops[('pytorch', 'numRepeats')]
ops[('improve', '%BatchTime')] = (ops[('improve', 'repsTimeDiff')] / batchTime) * 100
display(ops.sort_values(('improve', '%BatchTime'), ascending=False).head(50))

In [ ]:
ops.to_csv('/root/MLPerf-mGPU/image_classification/resnet50v15-compare-cuda10_1-rocm3_0-norm-distinctOps.csv')